In [1]:
import os
os.environ['HF_HOME'] = 'huggingface'
os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = 'True'
import math
from datasets import Audio, Dataset, DatasetDict
from transformers import Wav2Vec2Processor, Wav2Vec2ConformerForCTC, TrainingArguments, Trainer
import torch
from torch.utils.data.dataloader import DataLoader
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import numpy as np
import evaluate
import pandas as pd
from sklearn.model_selection import train_test_split
from pytorch_optimizer import Ranger21

In [2]:
model_name= 'facebook/wav2vec2-conformer-rel-pos-large-960h-ft'

In [3]:
processor = Wav2Vec2Processor.from_pretrained(model_name)

In [4]:
ds_df = pd.read_csv('Train.csv')
train_df, val_df = train_test_split(ds_df, test_size=0.2)

In [5]:
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(train_df)

In [6]:
train_ds = train_ds.cast_column("path", Audio(sampling_rate=16000))
val_ds = val_ds.cast_column("path", Audio(sampling_rate=16000))
ds = DatasetDict({'train': train_ds, 'val': val_ds})

def prepare_dataset(batch):
    model_name = 'facebook/wav2vec2-conformer-rel-pos-large-960h-ft'
    from transformers import Wav2Vec2Processor
    processor = Wav2Vec2Processor.from_pretrained(model_name)
    audio = batch["path"]
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["annotation"])
    batch["input_length"] = len(batch["input_values"][0])
    return batch

ds = ds.map(prepare_dataset, num_proc=4)

num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.


Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.


Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

In [7]:
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"][0]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

        labels_batch = self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding="longest")

In [8]:
wer = evaluate.load("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    return {"wer": wer.compute(predictions=pred_str, references=label_str)}

In [9]:
model = Wav2Vec2ConformerForCTC.from_pretrained(
    model_name,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id)

In [10]:
model.freeze_feature_encoder()

In [12]:
per_gpu_bs = 8
effective_bs = 32
training_args = TrainingArguments(
    output_dir="checkpoints",
    overwrite_output_dir =True,
    per_device_train_batch_size=per_gpu_bs,
    gradient_accumulation_steps=math.ceil(effective_bs/per_gpu_bs),
    learning_rate=1e-4,
    num_train_epochs=100,
    gradient_checkpointing=False,
    fp16=True,
    group_by_length=True,
    evaluation_strategy="epoch",
    save_strategy='no',  # epoch
    save_safetensors=True,
    per_device_eval_batch_size=4,
    save_steps=1,
    eval_steps=1,
    logging_steps=100,
    save_total_limit=3,
    lr_scheduler_type='cosine',
    load_best_model_at_end=False,  # True
    adam_beta1=0.9,
    adam_beta2=0.98,  # follow fairseq fintuning config
    warmup_ratio=0.22, # follow Ranger21
    weight_decay=1e-4,  # follow Ranger21
    metric_for_best_model="wer",
    greater_is_better=False,
    fp16_full_eval=True,
    torch_compile=os.name!='nt',
    report_to=['tensorboard'],
    torch_compile_backend='inductor' if os.name != 'nt' else None)

In [13]:
class CTCTrainer(Trainer):
    def training_step(self, model: torch.nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)
        loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if os.name != 'nt':
            accelerator.backward(self.scaler.scale(loss))
        else:
            self.scaler.scale(loss).backward()
        return loss.detach()

In [14]:
if os.name != 'nt':
    from accelerate import Accelerator
    accelerator = Accelerator(mixed_precision='fp16', dynamo_backend='eager')  # FP8 needs transformer_engine package which is only on Linux with Hopper GPUs

In [15]:
def tri_stage_schedule(epoch: int, max_epoch = training_args.num_train_epochs, stage_ratio = [0.1, 0.4, 0.5], peak_lr = training_args.learning_rate, initial_lr_scale=0.01, final_lr_scale=0.05):
    """https://github.com/facebookresearch/fairseq/blob/5ecbbf58d6e80b917340bcbf9d7bdbb539f0f92b/fairseq/optim/lr_scheduler/tri_stage_lr_scheduler.py#L51"""
    assert sum(stage_ratio) == 1
    current_ratio = epoch / max_epoch
    if current_ratio < stage_ratio[0]:  # linear warmup
        lrs = torch.linspace(initial_lr_scale * peak_lr, peak_lr, int(stage_ratio[0] * max_epoch))
        return lrs[epoch]
    elif stage_ratio[0] <= current_ratio <= stage_ratio[1]:  # constant
        return peak_lr
    else:  # exponential decay
        decay_factor = -math.log(final_lr_scale) / (stage_ratio[2] * max_epoch)
        return peak_lr * math.exp(-decay_factor * stage_ratio[2] * max_epoch)

In [16]:
max_steps = math.ceil(training_args.num_train_epochs * len(ds['train']) / training_args.gradient_accumulation_steps / min(training_args.per_device_train_batch_size, len(ds['train'])))
# optimizer = Ranger21(model.parameters(), num_iterations=max_steps)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-8, foreach=False)  # https://github.com/facebookresearch/fairseq/blob/main/examples/wav2vec/config/finetuning/base_960h.yaml
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_steps)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=tri_stage_schedule)  # following FAIR finetuning settings
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda x: x)  # constant LR, stays same throughout, for Ranger21

trainer = CTCTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds['val'],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # optimizers=(optimizer, scheduler),
)
if os.name != 'nt':  # windows does not support torch.compile yet
    trainer.model_wrapped, trainer.optimizer, trainer.lr_scheduler = accelerator.prepare(trainer.model_wrapped, trainer.optimizer, trainer.lr_scheduler)
trainer.train()
if os.name != 'nt':
    accelerator.wait_for_everyone()

C:\Program Files\Python39\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\alien\AppData\Local\Temp\ipykernel_5312\3519010244.py:20 in <module>                    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\alien\\AppData\\Local\\Temp\\ipykernel_5312\\3519010244.py'                          │
│                                                                                                  │
│ C:\Program Files\Python39\lib\site-packages\transformers\trainer.py:1664 in train                │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ C:\Program Files\Python39\lib\site-packages\transformers\trainer.py:1940 in _inner_training_loop │
│                                                                                                  │
│   1937 │   │   │   │   │   with model.no_sync():                                                 │
│   1938 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1939 │   │   │   │   else:                                                                     │
│ ❱ 1940 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1941 │   │   │   │                                                                             │
│   1942 │   │   │   │   if (                                                                      │
│   1943 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ C:\Users\alien\AppData\Local\Temp\ipykernel_5312\3610558995.py:23 in training_step               │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\alien\\AppData\\Local\\Temp\\ipykernel_5312\\3610558995.py'                          │
│                                                                                                  │
│ C:\Program Files\Python39\lib\site-packages\transformers\trainer.py:2767 in compute_loss         │
│                                                                                                  │
│   2764 │   │   │   labels = inputs.pop("labels")                                                 │
│   2765 │   │   else:                                                                             │
│   2766 │   │   │   labels = None                                                                 │
│ ❱ 2767 │   │   outputs = model(**inputs)                                                         │
│   2768 │   │   # Save past state if it exists                                                    │
│   2769 │   │   # TODO: this needs to be fixed and made cleaner later.                            │
│   2770 │   │   if self.args.past_index >= 0:               

In [28]:
# if os.name != 'nt':
#     trainer.model_wrapped = accelerator.unwrap_model(trainer.model_wrapped)
trainer.save_model('wav2vec2-conformer')
processor.tokenizer.save_pretrained('wav2vec2-conformer')

('wav2vec2-conformer\\tokenizer_config.json',
 'wav2vec2-conformer\\special_tokens_map.json',
 'wav2vec2-conformer\\vocab.json',
 'wav2vec2-conformer\\added_tokens.json')

In [1]:
from transformers import Wav2Vec2ConformerForCTC
model = Wav2Vec2ConformerForCTC.from_pretrained('wav2vec2-conformer')

You are using a model of type wav2vec2 to instantiate a model of type wav2vec2-conformer. This is not supported for all configurations of models and can yield errors.
C:\Program Files\Python39\lib\site-packages\transformers\modeling_utils.py:433: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(checkpoint_file, framework="pt") as f:


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\alien\AppData\Local\Temp\ipykernel_6308\2135182543.py:2 in <module>                     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\alien\\AppData\\Local\\Temp\\ipykernel_6308\\2135182543.py'                          │
│                                                                                                  │
│ C:\Program Files\Python39\lib\site-packages\transformers\modeling_utils.py:2777 in               │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   2774 │   │   │   │   mismatched_keys,                                                          │
│   2775 │   │   │   │   offload_index,                                                            │
│   2776 │   │   │   │   error_msgs,                                                               │
│ ❱ 2777 │   │   │   ) = cls._load_pretrained_model(                                               │
│   2778 │   │   │   │   model,                                                                    │
│   2779 │   │   │   │   state_dict,                                                               │
│   2780 │   │   │   │   loaded_state_dict_keys,  # XXX: rename?                                   │
│                                                                                                  │
│ C:\Program Files\Python39\lib\site-packages\transformers\modeling_utils.py:3003 in               │
│ _load_pretrained_model                                                                           │
│                                                                                                  │
│   3000 │   │   │   model_to_load = getattr(model, cls.base_model_prefix)                         │
│   3001 │   │   │   base_model_expected_keys = list(model_to_load.state_dict().keys())            │
│   3002 │   │   │   if any(key in expected_keys_not_prefixed and key not in base_model_expected_  │
│ ❱ 3003 │   │   │   │   raise ValueError(                                                         │
│   3004 │   │   │   │   │   "The state dictionary of the model you are trying to load is corrupt  │
│   3005 │   │   │   │   │   "properly saved?"                                                     │
│   3006 │   │   │   │   )                                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: The state dictionary of the model you are trying to load is corrupted. Are you sure it was properly 
saved?

In [15]:
# Infer
from transformers import pipeline
transcriber = pipeline("automatic-speech-recognition", model="wav2vec2-conformer")
def predict(audio_file):
    r = transcriber(audio_file)['text'].upper()
    if "'" in r:
        print(audio_file, f'has \' in {r}, removing')
        r = r.replace("'", '')  # Tokenizer includes "'" but TIL dataset does not
    return r

C:\Program Files\Python39\lib\site-packages\transformers\modeling_utils.py:433: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(checkpoint_file, framework="pt") as f:


In [16]:
test_ds = pd.read_csv('Test_Advanced.csv')
test_ds['annotation'] = test_ds['path'].map(predict)
test_ds.to_csv('Test_Advanced.csv', index=False)